# 👋 Olá, estou muito feliz em vê-lo aqui. 
Presumo que você está procurando uma maneira fácil de criar seu avatar com uma inteligência artificial. Bem, é para isso que estou aqui, vou lhe guiar nessa jornada.

Abaixo, está o passo a passo de como você pode criar as imagens, siga-as, e caso ocorra algum problema, você tenha uma dúvida, ou ainda uma melhoria, siga para [esse repositório do Github](https://github.com/Dpbm/dreambooth-tutorial) e abra uma `ISSUE`, assim que possível a comunidade irá te ajudar!

Pronto para começar?

# 1 - HuggingFace

![HuggingFace ícone](https://github.com/Dpbm/dreambooth-tutorial/blob/main/notebooks/assets/hugging-face-image.svg)

O huggingFace, é uma plataforma, da qual pessoas engajadas no universo de inteligência artificial podem compartilhar pesquisas, [modelos](https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial), etc.\
Além de possibilitar pessoas que não são da área, a utilizarem essas IAs de uma forma fácil através de uma [API](https://pt.wikipedia.org/wiki/Interface_de_programa%C3%A7%C3%A3o_de_aplica%C3%A7%C3%B5es).

Aqui utilizaremos para nos comunicarmos com um modelo chamado [Stable Diffusion](https://huggingface.co/runwayml/stable-diffusion-v1-5).

1. Para começar, entre em [huggingface.co](https://huggingface.co) e [crie uma conta](https://huggingface.co/join), caso você não possua uma;
2. Em seguida, o HuggingFace irá lhe enviar um e-mail para verificar sua conta, entre nele e clique no link que lhe enviaram (lembre de olhar também a caixa de spam);
3. Com sua conta verificada, [clique aqui](https://huggingface.co/settings/tokens);
4. Clique em `New Token`(ou nova token, dependendo do idioma que você está utilizando);
5. O seguinte campo abrirá em sua tela:

![campo para criar o token](https://github.com/Dpbm/dreambooth-tutorial/blob/main/notebooks/assets/token_field.png)

Em `name` coloque um nome para sua token, recomendo colocar um nome sem caracteres especiais (como ponto, vírgula, espaço, etc.) e sem acentos, isso pode gerar erros mais tarde.\
Em `role` mude para `write` (ou escrita, dependendo do seu idioma do navegador).

6. Após criada a token, `anote o nome da token em algum local` e depois clique em `copiar` e `também anote a token gerada em algum local`

![copiar token](https://github.com/Dpbm/dreambooth-tutorial/blob/main/notebooks/assets/copy_token.png)
 

# 2 - Suas fotos

Agora, você precisa tirar diversas fotos de você, `pelo menos 20`, com uma boa qualidade.\
Lembre de tirar fotos em diversos angulos, posições, com roupas diferentes, acessórios diferentes, etc. Tudo para o modelo te entender melhor. 

Após tirá-las, você precisa garantir que elas tenham `512x512px`. Você pode usar diversas aplicações para fazer isso, mas a mais fácil é o Birme, clique [aqui](https://www.birme.net/?target_width=512&target_height=512) e recorte suas imagens. 

# 3 - Google Colab

Agora que você está de volta no google colab, você precisa verificar se o ambiente está configurado para o uso de `GPU`. Para verificar isso vá em `Runtime > Change Runtime Type` e modifique o `Hardware accelerator` para `GPU`.

![GPU no google colab runtime](https://github.com/Dpbm/dreambooth-tutorial/blob/main/notebooks/assets/GPU.png)

Se algum erro acontecer, tente rodar o notebook mais tarde. Caso você queira executar no mesmo instante dê uma olhada nos planos PRO do google colab, dos quais estão disponíveis [nesse link](https://colab.research.google.com/signup)

# 4 - Rodando os códigos

Agora precisamos começar a executar os códigos. 

Fique atento para erros, caso algum ocorra, vá para o [repositório do projeto no github](https://github.com/Dpbm/dreambooth-tutorial/issues) e veja se há alguma issue aberta discutindo o erro que está acontecendo para você, se não houver, fique à vontade para abrir uma nova issue.
Não esqueça de sempre que possível ajudar nossa comunidade, mesmo que você não esteja familiarizado com tecnologia e programação, não se preocupe, interaja com os outros através das issues e seja feliz :)

Depois desse recado, vamos à execução

In [ ]:
#@markdown 🏃‍♂️ Rode esta célula para instalar as dependencias necessárias
!rm -rf train_dreambooth.py convert_diffusers_to_original_stable_diffusion.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes gradio natsort
%pip install -q https://github.com/Dpbm/dreambooth-tutorial/releases/download/07ba3f3/xformers-0.0.15.dev0+07ba3f3.d20221208-cp38-cp38-linux_x86_64.whl

In [ ]:
#@markdown 🏃‍♂️ Adcione o nome da token e a token, respectivamente, abaixo

from google.colab import drive
import os
from pathlib import Path

def createFolder(path):
  if(not os.path.exists(path)):
    print(f"[*] Criando a pasta {path}...")
    os.mkdir(path)

def createConceptsList(path):
  import json

  concepts = [
      {
          "instance_prompt":      HUGGINGFACE_TOKEN_NAME,
          "class_prompt":         "photo of a person",
          "instance_data_dir":    INSTANCE_DATA_FOLDER,
          "class_data_dir":       f"{DATA_FOLDER}person"
      }
  ]


  print(f"[*] Criando o arquivo {path}...")
  with open(path, "w") as file:
      json.dump(concepts, file, indent=4)

def createHuggingFaceToken(path, token):
  token_file_path = os.path.join(path, 'token')

  if(os.path.exists(token_file_path)): return

  print(f"[*] Salvando a token do HuggingFace em {token_file_path}...")

  with open(token_file_path, "w") as token_file:
    token_file.write(token)

HOME=Path.home()
HUGGING_FACE_TOKEN_FOLDER=f'{HOME}/.huggingface'
createFolder(HUGGING_FACE_TOKEN_FOLDER)

HUGGINGFACE_TOKEN_NAME = "" #@param {type:"string"}
HUGGINGFACE_TOKEN = "" #@param {type:"string"}

createHuggingFaceToken(HUGGING_FACE_TOKEN_FOLDER, HUGGINGFACE_TOKEN)

MODEL_NAME = "runwayml/stable-diffusion-v1-5"

#@markdown Após adicionar os dados e rodar a célula, dê permissão para acessar o seu google drive

drive.mount('/content/drive')

OUTPUT_DIR = "/content/drive/MyDrive/stable_diffusion_weights/zwx"
createFolder(OUTPUT_DIR)

DATA_FOLDER = "/content/drive/MyDrive/data/"
INSTANCE_DATA_FOLDER = f"{DATA_FOLDER}{HUGGINGFACE_TOKEN_NAME}"
createFolder(DATA_FOLDER)
createFolder(INSTANCE_DATA_FOLDER)

CONPECTS_LIST_JSON = "/content/drive/MyDrive/concepts_list.json"
createConceptsList(CONPECTS_LIST_JSON)

WEIGHTS_DIR = "/content/drive/MyDrive/stable_diffusion_weights/zwx/800"

OUTPUT_GENERATED_IMAGES = "/content/drive/MyDrive/imagens_geradas/"
createFolder(OUTPUT_GENERATED_IMAGES)


In [ ]:
#@markdown 🏃‍♂️ Rode essa célula e faça upload das suas imagens

from google.colab import files
import os
import shutil

uploaded = files.upload()
for filename in uploaded.keys():
  path = os.path.join(INSTANCE_DATA_FOLDER, filename)
  print(f"[*] movendo {filename} para {path}...")
  shutil.move(filename, path)

In [ ]:
#@markdown 🏃‍♂️ Rode essa célula para treinar o modelo. Isso pode demorar um pouco, tenha paciência

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=800 \
  --save_interval=10000 \
  --save_sample_prompt=$HUGGINGFACE_TOKEN_NAME \
  --concepts_list=$CONPECTS_LIST_JSON

In [ ]:
#@markdown 🏃‍♂️ Rode essa célula para preparar o ambiente para executar a IA

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = torch.Generator(device='cuda')
seed = 52362
g_cuda.manual_seed(seed)

In [ ]:
#@markdown Comece a criar as imagens, lembre-se de colocar o nome da sua token no texto.\
#@markdown exemplo:\ 
#@markdown *nome da token: eu*\
#@markdown *texto: eu wearing a black hat*

import uuid

prompt = "" #@param {type:"string"}

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=512,
        width=512,
        negative_prompt="",
        num_images_per_prompt=4,
        num_inference_steps=24,
        guidance_scale=7.5,
        generator=g_cuda
    ).images

for img in images:
  image_path = f"{OUTPUT_GENERATED_IMAGES}{uuid.uuid4()}.png"
  print(f"[*] Salvando imagem {image_path}...")
  img.save(image_path)
  
  display(img)

In [ ]:
#@markdown 🏃‍♂️ Rode essa célula para terminar as execuções e finalizar o ambiente
exit()

[Versão original do repositório](https://github.com/ShivamShrirao/diffusers) [by ShivamShrirao](https://github.com/ShivamShrirao)  